In [34]:
import os
import pickle
import pandas as pd


# ============================================================
# 1. BASE DIRECTORY
# ============================================================

BASE_DIR = os.path.join(
    os.path.expanduser("~"),
    "Downloads",
    "FLIPOSE_DATA",
    "V_2.2",
    "Models_predictions"
)

print("Using Base Directory:\n", BASE_DIR)
print("\nFiles in directory:", os.listdir(BASE_DIR))


# ============================================================
# 2. DIRECT MODEL AREAS (NO MAPPING)
# ============================================================

DIRECT_MODEL_AREAS = {
    "Al Barsha South Fourth",
    "Business Bay",
    "Al Merkadh",
    "Burj Khalifa",
    "Hadaeq Sheikh Mohammed Bin Rashid",
    "Al Khairan First",
    "Wadi Al Safa 5",
    "Al Thanyah Fifth",
    "Al Barshaa South Third",
    "Jabal Ali First",
    "Madinat Al Mataar",
    "Madinat Dubai Almelaheyah",
    "Me'Aisem First",
    "Al Hebiah Fourth",
    "Al Barsha South Fifth",
    "Al Hebiah First",
    "Nadd Hessa",
    "Palm Jumeirah",
    "Al Barshaa South Second",
    "Al Yelayiss 2",
    "Al Warsan First"
}


# ============================================================
# 3. PROXY / GROUP MAPPING
# ============================================================

PROXY_MAPPING = {

    # proxy1
    "Al Barsha First": "proxy1",
    "Al Barshaa South Second": "proxy1",
    "Al Hebiah Second": "proxy1",
    "Al Hebiah Sixth": "proxy1",
    "Al Hebiah Third": "proxy1",
    "Madinat Hind 4": "proxy1",
    "Wadi Al Safa 3": "proxy1",
    "Wadi Al Safa 4": "proxy1",
    "Wadi Al Safa 7": "proxy1",

    # proxy2
    "Bukadra": "proxy2",
    "Hadaeq Sheikh Mohammed Bin Rashid": "proxy2",
    "Ras Al Khor Industrial First": "proxy2",
    "Jumeirah First": "proxy2",
    "Palm Deira": "proxy2",
    "Al Khairan First": "proxy2",

    # proxy3
    "Al Thanyah Third": "proxy3",
    "Jabal Ali Industrial Second": "proxy3",

    # groups
    "Al Kifaf": "G1",
    "Warsan Fourth": "G3",
    "Jabal Ali": "G3",
    "Zaabeel Second": "G4",
    "Zaabeel First": "G4"
}


# ============================================================
# 4. LOAD TRAIN COLUMNS
# ===========================================================
COL_DIR = os.path.join(BASE_DIR,
"Trained Columns"
)
def load_columns(model_key):
    for f in os.listdir(COL_DIR):
        if f.lower() == f"trained_columns_{model_key}.pkl".lower():
            with open(os.path.join(COL_DIR, f), "rb") as file:
                return pickle.load(file)

    raise FileNotFoundError(f"trained_columns_{model_key}.pkl not found")


# ============================================================
# 5. LOAD MODEL
# ============================================================

MODEL_DIR = os.path.join(BASE_DIR,
"Models"
)
def load_model(model_key):
    for f in os.listdir(MODEL_DIR):
        if f.lower() == f"rf_model_{model_key}.pkl".lower():
            with open(os.path.join(MODEL_DIR, f), "rb") as file:
                return pickle.load(file)

    raise FileNotFoundError(f"rf_model_{model_key}.pkl not found")


# ============================================================
# 6. MAIN PREDICTION FUNCTION
# ============================================================

def predict_with_proxy(input_data, forecast_df, historic_df):

    area = input_data["area_name"]

    # --------------------------------------------------------
    # Decide model key
    # --------------------------------------------------------
    if area in DIRECT_MODEL_AREAS:
        model_key = area
        model_type = "direct"
    elif area in PROXY_MAPPING:
        model_key = PROXY_MAPPING[area]
        model_type = "proxy"
    else:
        raise ValueError(f"No model found for area: {area}")

    # --------------------------------------------------------
    # Load model & columns
    # --------------------------------------------------------
    train_columns = load_columns(model_key)
    model = load_model(model_key)

    # --------------------------------------------------------
    # Feature engineering
    # --------------------------------------------------------
    categorical_features = [
        "reg_type_en",
        "rooms_en",
        "land_type_en",
        "floor_bin",
        "developer_cat",
        "project_cat"
    ]

    binary_features = [
        "has_parking",
        "swimming_pool",
        "balcony",
        "elevator",
        "metro"
    ]

    continuous_features = ["procedure_area"]

    temp = pd.DataFrame([input_data])
    temp = temp[categorical_features + binary_features + continuous_features]

    temp = pd.get_dummies(temp, columns=categorical_features, drop_first=False)

    for col in train_columns:
        if col not in temp.columns:
            temp[col] = 0

    temp = temp[train_columns]

    # --------------------------------------------------------
    # Predict
    # --------------------------------------------------------
    predicted_price = model.predict(temp)[0]
    print("Raw Model Prediction:", predicted_price)

    # --------------------------------------------------------
    # DIRECT MODEL → RETURN ONLY PREDICTION
    # --------------------------------------------------------
    # --------------------------------------------------------
    # APPLY FORECAST & HISTORIC IF DATA EXISTS
    # --------------------------------------------------------
    
    has_forecast = model_key in forecast_df["area"].values
    has_historic = model_key in historic_df["area"].values
    
    # If no time-series data → return simple prediction
    if not has_forecast and not has_historic:
        return pd.DataFrame({
            "area": [area],
            "predicted_price": [predicted_price]
        })
    
    # --------------------------------------------------------
    # FORECAST DATA
    # --------------------------------------------------------
    gf = forecast_df[forecast_df["area"] == model_key].copy()
    
    if not gf.empty:
        gf["median_price"] = predicted_price * gf["growth_factor"]
    
    # --------------------------------------------------------
    # HISTORIC DATA
    # --------------------------------------------------------
    historic = historic_df[historic_df["area"] == model_key].copy()
    
    if not historic.empty:
        historic.loc[historic.index[-1], "median_price"] = predicted_price
    
    # --------------------------------------------------------
    # FINAL TIME SERIES
    # --------------------------------------------------------
    final_df = pd.concat([historic, gf], ignore_index=True)
    
    final_df = final_df[["month", "median_price", "area"]]
    final_df["area"] = area  # show requested area name
    final_df["month"] = pd.to_datetime(final_df["month"], format="%d-%m-%Y")
    
    final_df = (
        final_df
        .sort_values("month")
        .reset_index(drop=True)
    )
    
    return final_df



Using Base Directory:
 C:\Users\pooja\Downloads\FLIPOSE_DATA\V_2.2\Models_predictions

Files in directory: ['.ipynb_checkpoints', 'forecast_df.csv', 'historic_df.csv', 'Models', 'Models_predictions.zip', 'predict_price.py', 'price_api', 'Testing_model.ipynb', 'testing_model.py', 'Trained Columns', 'V_2.2_inputs.csv']


    # --------------------------------------------------------
    # Step 3: One-hot encode input
    # --------------------------------------------------------
    temp = pd.DataFrame([input_data])
    temp = pd.get_dummies(temp)

    # Add missing columns
    for col in train_columns:
        if col not in temp.columns:
            temp[col] = 0

    # Align strictly with model columns
    temp = temp[train_columns]


In [35]:
forecast_df = pd.read_csv("forecast_df.csv")
historic_df = pd.read_csv("historic_df.csv")

DIRECT_MODEL_AREAS = {
    "Al Barsha South Fourth",
    "Business Bay",
    "Al Merkadh",
    "Burj Khalifa",
    "Hadaeq Sheikh Mohammed Bin Rashid",
    "Al Khairan First",
    "Wadi Al Safa 5",
    "Al Thanyah Fifth",
    "Al Barshaa South Third",
    "Jabal Ali First",
    "Madinat Al Mataar",
    "Madinat Dubai Almelaheyah",
    "Me'Aisem First",
    "Al Hebiah Fourth",
    "Al Barsha South Fifth",
    "Al Hebiah First",
    "Nadd Hessa",
    "Palm Jumeirah",
    "Al Barshaa South Second",
    "Al Yelayiss 2",
    "Al Warsan First"
}


proxy_mapping = {

    # =====================================================
    # INPUT AREAS → PROXY 1
    # =====================================================
    "Al Barsha First": "proxy1",
    "Al Barshaa South Second": "proxy1",
    "Al Hebiah Second": "proxy1",
    "Al Hebiah Sixth": "proxy1",
    "Al Hebiah Third": "proxy1",
    "Madinat Hind 4": "proxy1",
    "Wadi Al Safa 3": "proxy1",
    "Wadi Al Safa 4": "proxy1",
    "Wadi Al Safa 7": "proxy1",

    # =====================================================
    # INPUT AREAS → PROXY 2
    # =====================================================
    "Bukadra": "proxy2",
    "Hadaeq Sheikh Mohammed Bin Rashid": "proxy2",
    "Ras Al Khor Industrial First": "proxy2",
    "Jumeirah First": "proxy2",
    "Palm Deira": "proxy2",
    "Al Khairan First": "proxy2",

    # =====================================================
    # INPUT AREAS → PROXY 3
    # =====================================================
    "Al Thanyah Third": "proxy3",
    "Jabal Ali Industrial Second": "proxy3",

    # =====================================================
    # INPUT AREAS → G1
    # =====================================================
    "Al Kifaf": "G1",

    # =====================================================
    # INPUT AREAS → G3
    # =====================================================
    "Warsan Fourth": "G3",
    "Jabal Ali": "G3",

    # =====================================================
    # INPUT AREAS → G4
    # =====================================================
    "Zaabeel Second": "G4",
    "Zaabeel First": "G4"
}


In [40]:
input_data = {
    "area_name": "Al Hebiah First",

    # categorical
    "reg_type_en": "off-plan",
    "rooms_en": "1B/R",
    "land_type_en": "Commercial",
    "floor_bin": "11-20",
    "developer_cat": "Grade 2",
    "project_cat": "Mid_Rise",

    # binary
    "has_parking": 1,
    "swimming_pool": 0,
    "balcony": 0,
    "elevator": 1,
    "metro": 1,

    # continuous
    "procedure_area": 20
}



In [41]:
final_output = predict_with_proxy(
    input_data,
    forecast_df,
    historic_df
)

print("\nFINAL DF:\n", final_output.tail(15))

Raw Model Prediction: 13756.993107286125

FINAL DF:
         month  median_price             area
67 2025-08-01  15195.730000  Al Hebiah First
68 2025-09-01  15894.080000  Al Hebiah First
69 2025-10-01  13756.993107  Al Hebiah First
70 2025-12-01  13665.902804  Al Hebiah First
71 2025-12-01  13718.178577  Al Hebiah First
72 2026-01-01  13985.164091  Al Hebiah First
73 2026-01-01  13584.976507  Al Hebiah First
74 2026-02-01  13427.018978  Al Hebiah First
75 2026-02-01  14254.425631  Al Hebiah First
76 2026-03-01  14258.842548  Al Hebiah First
77 2026-03-01  13288.836295  Al Hebiah First
78 2026-04-01  13301.649201  Al Hebiah First
79 2026-04-01  14257.831959  Al Hebiah First
80 2026-05-01  13468.491972  Al Hebiah First
81 2026-05-01  14391.398440  Al Hebiah First


C:\Users\pooja\anaconda3\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\pooja\anaconda3\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [29]:
import sklearn
print(sklearn.__version__)


1.6.1
